### Creating a Siamese model using Trax

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential
from tensorflow import math
import numpy

numpy.random.seed(10)

2024-01-02 09:23:33.798193: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-02 09:23:33.838451: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-02 09:23:33.838503: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-02 09:23:33.839901: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-02 09:23:33.848322: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-02 09:23:33.849618: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
tensor = numpy.random.random((2, 5))
print(f'The tensor is of type: {type(tensor)}\n\nAnd looks like this:\n\n {tensor}')

The tensor is of type: <class 'numpy.ndarray'>

And looks like this:

 [[0.77132064 0.02075195 0.63364823 0.74880388 0.49850701]
 [0.22479665 0.19806286 0.76053071 0.16911084 0.08833981]]


To create a `Siamese` model you will first need to create a LSTM model. For this you can stack layers using the`Sequential` model. To retrieve the output of both branches of the Siamese model, you can concatenate results using the `Concatenate` layer. You should be familiar with the following layers (notice each layer can be clicked to go to the docs):
   - [`Sequential`](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) groups a linear stack of layers into a [`tf.keras.Model`](https://www.tensorflow.org/api_docs/python/tf/keras/Model)
   - [`Embedding`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) Maps positive integers into vectors of fixed size. It will have shape (vocabulary length X dimension of output vectors). The dimension of output vectors (called `model_dimension`in the code) is the number of elements in the word embedding. 
   - [`LSTM`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM) The Long Short-Term Memory (LSTM) layer. The number of units should be specified and should match the number of elements in the word embedding. 
   - [`GlobalAveragePooling1D`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GlobalAveragePooling1D) Computes global average pooling, which essentially takes the mean across a desired axis. GlobalAveragePooling1D uses one tensor axis to form groups of values and replaces each group with the mean value of that group. 
   - [`Lambda`](https://trax-ml.readthedocs.io/en/latest/trax.layers.html#trax.layers.base.Fn)  Layer with no weights that applies the function f, which should be specified using a lambda syntax. You will use this layer to apply normalization with the function
        - `tfmath.l2_normalize(x)`
        
- [`Concatenate`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Concatenate) Layer that concatenates a list of inputs. This layer will concatenate the normalized outputs of each LSTM into a single output for the model.
- [`Input`](https://www.tensorflow.org/api_docs/python/tf/keras/Input): it is used to instantiate a Keras tensor.. Remember to set correctly the dimension and type of the input, which are batches of questions. 


Putting everything together the Siamese model will look like this:

In [3]:
vocab_size = 500
model_dimension = 128

# Define the LSTM model
LSTM = Sequential()
LSTM.add(layers.Embedding(input_dim=vocab_size, output_dim=model_dimension))
LSTM.add(layers.LSTM(units=model_dimension, return_sequences = True))
LSTM.add(layers.AveragePooling1D())
LSTM.add(layers.Lambda(lambda x: math.l2_normalize(x)))

input1 = layers.Input((vocab_size,))
input2 = layers.Input((vocab_size,))

# Concatenate two LSTMs together
conc = layers.Concatenate(axis=1)((LSTM(input1), LSTM(input2)))
    

# Use the Parallel combinator to create a Siamese model out of the LSTM 
Siamese = Model(inputs=(input1, input2), outputs=conc)

# Print the summary of the model
Siamese.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 500)]                0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 500)]                0         []                            
                                                                                                  
 sequential (Sequential)     (None, None, 128)            195584    ['input_1[0][0]',             
                                                                     'input_2[0][0]']             
                                                                                                  
 concatenate (Concatenate)   (None, 500, 128)             0         ['sequential[0][0]',      

In [4]:
def show_layers(model, layer_prefix):
    print(f"Total layers: {len(model.layers)}\n")
    for i in range(len(model.layers)):
        print('========')
        print(f'{layer_prefix}_{i}: {model.layers[i]}\n')

print('Siamese model:\n')
show_layers(Siamese, 'Parallel.sublayers')

print('Detail of LSTM models:\n')
show_layers(LSTM, 'Serial.sublayers')

Siamese model:

Total layers: 4

Parallel.sublayers_0: <keras.src.engine.input_layer.InputLayer object at 0x7fd274390050>

Parallel.sublayers_1: <keras.src.engine.input_layer.InputLayer object at 0x7fd27437c050>

Parallel.sublayers_2: <keras.src.engine.sequential.Sequential object at 0x7fd30f1e3810>

Parallel.sublayers_3: <keras.src.layers.merging.concatenate.Concatenate object at 0x7fd274319a10>

Detail of LSTM models:

Total layers: 4

Serial.sublayers_0: <keras.src.layers.core.embedding.Embedding object at 0x7fd2774671d0>

Serial.sublayers_1: <keras.src.layers.rnn.lstm.LSTM object at 0x7fd2774e19d0>

Serial.sublayers_2: <keras.src.layers.pooling.average_pooling1d.AveragePooling1D object at 0x7fd277f40710>

Serial.sublayers_3: <keras.src.layers.core.lambda_layer.Lambda object at 0x7fd274318ed0>

